# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

c:\Users\shail\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:

import os
from glob import glob

# PRICE_DATA directory 
price_data_dir = os.getenv("PRICE_DATA")
if not price_data_dir:
    raise RuntimeError("PRICE_DATA not set. Did you run %dotenv?")

#Glob for every .parquet (recursively)
all_files = glob(os.path.join(price_data_dir, "**", "*.parquet"), recursive=True)

#Take only the first 60
sample_files = all_files[:60]

#check
print(f"Found {len(all_files)} total parquet files under {price_data_dir}")
print(f"Using first 60 → sample_files list length: {len(sample_files)}")
for path in sample_files:
    print("  ", path)





Found 2897 total parquet files under ../../05_src/data/prices/
Using first 60 → sample_files list length: 60
   ../../05_src/data/prices\ACHV\ACHV_1995\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_1995\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_1996\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_1996\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_1997\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_1997\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_1998\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_1998\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_1999\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_1999\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_2000\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_2000\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_2001\part.0.parquet
   ../../05_src/data/prices\ACHV\ACHV_2001\part.1.parquet
   ../../05_src/data/prices\ACHV\ACHV_2002\part.0.parquet
   ../../05_src/data/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:


# 1. Read first 60 files
ddf = dd.read_parquet(sample_files, engine="pyarrow")

# 2. Rename “Adj Close” to a Python‐friendly name
ddf = ddf.rename(columns={"Adj Close": "Adj_Close"})

# 3. Build your lag, returns and hi_lo_range features
dd_feat = (
    ddf
    .assign(
        Close_lag_1     = lambda df: df.groupby("ticker")["Close"].shift(1),
        Adj_Close_lag_1 = lambda df: df.groupby("ticker")["Adj_Close"].shift(1),
    )
    .assign(
        returns     = lambda df: (df["Close"] / df["Close_lag_1"]) - 1,
        hi_lo_range = lambda df: df["High"] - df["Low"],
    )
    .persist()
)

# 4. Inspect
print(dd_feat.head())





C:\Users\shail\AppData\Local\Temp\ipykernel_6524\3770542123.py:12: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  Close_lag_1     = lambda df: df.groupby("ticker")["Close"].shift(1),
C:\Users\shail\AppData\Local\Temp\ipykernel_6524\3770542123.py:13: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  Adj_Close_lag_1 = lambda df: df.groupby("ticker")["Adj_Close"].shift(1),


        Date     Open     High      Low    Close  Adj_Close  Volume    source  \
0 1995-10-12  14850.0  14850.0  13365.0  13612.5    13612.5     100  ACHV.csv   
1 1995-10-13  14355.0  14355.0  13365.0  14107.5    14107.5       0  ACHV.csv   
2 1995-10-16  14355.0  14355.0  13365.0  14355.0    14355.0       0  ACHV.csv   
3 1995-10-17  13612.5  14355.0  13365.0  13860.0    13860.0       0  ACHV.csv   
4 1995-10-18  14107.5  14850.0  13365.0  14107.5    14107.5       0  ACHV.csv   

  ticker  Year  Close_lag_1  Adj_Close_lag_1  returns  hi_lo_range  
0   ACHV  1995          NaN              NaN      NaN       1485.0  
1   ACHV  1995          NaN              NaN      NaN        990.0  
2   ACHV  1995          NaN              NaN      NaN        990.0  
3   ACHV  1995          NaN              NaN      NaN        990.0  
4   ACHV  1995          NaN              NaN      NaN       1485.0  


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# 1. Compute the Dask DataFrame into pandas
import pandas as pd

pd_df = dd_feat.compute()

# 2. Add a 10-day moving average 
pd_df["returns_ma_10"] = (
    pd_df
    .groupby("ticker")["returns"]          
    .rolling(window=10, min_periods=1)      
    .mean()                                
    .reset_index(level=0, drop=True)        
)

# 3. check
print(pd_df[["ticker", "Date", "returns", "returns_ma_10"]].head(15))




   ticker       Date  returns  returns_ma_10
0    ACHV 1995-10-12      NaN            NaN
1    ACHV 1995-10-13      NaN            NaN
2    ACHV 1995-10-16      NaN            NaN
3    ACHV 1995-10-17      NaN            NaN
4    ACHV 1995-10-18      NaN            NaN
5    ACHV 1995-10-19      NaN            NaN
6    ACHV 1995-10-20      NaN            NaN
7    ACHV 1995-10-23      NaN            NaN
8    ACHV 1995-10-24      NaN            NaN
9    ACHV 1995-10-25      NaN            NaN
10   ACHV 1995-10-26      NaN            NaN
11   ACHV 1995-10-27      NaN            NaN
12   ACHV 1995-10-30      NaN            NaN
13   ACHV 1995-10-31      NaN            NaN
14   ACHV 1995-11-01      NaN            NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
- Answer: Not nessary,Dask can supports groupby and rolling, however converting to pandas simplies for working in-memory for small dataseets.
+ Would it have been better to do it in Dask? Why?
- Answer Yes,rolling calculation helps to scale beyound memory and maintain parallelism.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.